In [6]:
import sys
sys.path.append('../')
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import DateFormatter
import matplotlib as mpl
import numpy as np

from utils import db

conn = db.connect()
pd.set_option('display.max_rows', 100)
# pd.set_option("display.max_rows", None, "display.max_columns", None)

# pd.read_sql("""SELECT * FROM stakes""", conn)

In [9]:
pd.read_sql("""SELECT fromaddress, COUNT(fromaddress)
FROM stakes
GROUP BY fromaddress
ORDER BY fromaddress DESC;""",conn)

,fromaddress,count
0,0xff6992fda98a9452e2eb0487e17533f428be6bdc,6
1,0xff53e3a27b7252148e0f1daf4a00261eee59177f,1
2,0xff4b28d2403ae65311cbdc0f0fac33f2d504648a,1
3,0xff2469082cc5bf7e626fbfa9c25f467411d84cc2,1
4,0xfefe61d43844bd8b034a11f33c88b5c9ed1caae1,4
...,...,...
810,0x01b2f6622b0815194b805df29b053c13a75f7363,1
811,0x00e56d0929070567b7d885ba90d79427f2882943,2
812,0x00dd07f525b82759313e9d0148b1344ca234dbc7,1
813,0x00b8cf5347b1d716af821df508f0c6a9037a367a,6
